In [ ]:
"""encrypt private key"""

import getpass
import json
from pathlib import Path
from web3 import Account

KEY_STORE_PASS = Path(".keystore.json")

private_key = getpass.getpass("Enter your private key: ")
my_account = Account.from_key(private_key)
password = getpass.getpass("Enter Your password: ")
encrypted_account = my_account.encrypt(password)

print(f"saving {KEY_STORE_PASS}...")
with open(KEY_STORE_PASS, "w+") as f:
    json.dump(encrypted_account, f, indent=2)


In [ ]:
"""Load Accounts"""

from web3 import Account
import getpass

with open(".keystore.json", 'r') as f:
    private_key = Account.decrypt(f.read(), getpass.getpass("Enter private key password: "))

account = Account.from_key(private_key)

In [ ]:
contract_name = "HateERC20"

In [ ]:
"""Build contract"""

import os, json
from solcx import compile_standard, install_solc


solc_version = install_solc("latest", solcx_binary_path="./solc/")

contract_sol_file_path = f"{contract_name}.sol"

with open(contract_sol_file_path, "r") as f:
    source_code = f.read()

# Compile contract
compiled = compile_standard(
    {
        "language": "Solidity",
        "sources": {contract_name + ".sol": {"content": source_code}},
        "settings": {"outputSelection": {"*": {"*": ["abi", "evm.bytecode.object"]}}},
    },
    solc_binary=f"solc/solc-v{solc_version}",
)

contract_output_dir = f"artifacts/"
os.makedirs(contract_output_dir, exist_ok=True)

with open(os.path.join(contract_output_dir, f"{contract_name}.json"), "w") as f:
    json.dump(compiled, f, indent=2)

In [ ]:
"""Load Contract ByteCode and ABI"""

import json

with open(f"artifacts/{contract_name}.json") as contract_compiled_file:
    compiled_contract = json.loads(contract_compiled_file.read())


abi = compiled_contract["contracts"][f"{contract_name}.sol"][contract_name]["abi"]
bytecode = compiled_contract["contracts"][f"{contract_name}.sol"][contract_name]["evm"][
    "bytecode"
]["object"]

bytecode, abi


In [ ]:
"""Connect to RPC async"""

from web3 import AsyncWeb3

async def get_new_ws_w3() -> AsyncWeb3:
    _ws_provider = AsyncWeb3.WebSocketProvider("ws://127.0.0.1:8545")
    w3_ws = AsyncWeb3(_ws_provider)
    await _ws_provider.connect()
    await w3_ws.is_connected(show_traceback=True)
    return w3_ws

w3 = AsyncWeb3(AsyncWeb3.AsyncHTTPProvider("http://127.0.0.1:8545"))
await w3.is_connected()

In [ ]:
"""Deploy contract"""
import asyncio
LoadedContract = w3.eth.contract(abi=abi, bytecode=bytecode)

transaction = await LoadedContract.constructor(account.address).build_transaction(
    {
        "from": account.address,
        "nonce": await w3.eth.get_transaction_count(account.address),
    }
)
signed_tx = w3.eth.account.sign_transaction(transaction, private_key)
tx_hash = await w3.eth.send_raw_transaction(signed_tx.raw_transaction)
await asyncio.sleep(.2)
receipt = await w3.eth.get_transaction_receipt(tx_hash)
deployed_addr = receipt["contractAddress"]
deployed_addr

In [ ]:
"""Create Contract Object from deployed address"""
deployed_addr = "" or deployed_addr
deployed_contract = w3.eth.contract(address=deployed_addr, abi=abi)

In [ ]:
await deployed_contract.functions.balanceOf(account=account.address).call()

In [ ]:
"""Mint"""

tx = await deployed_contract.functions.mint(to=account.address, amount=20000 * 10 ** 18).build_transaction(
    {
        "from": account.address,
        "nonce": await w3.eth.get_transaction_count(account.address),
    }
)
tx_hash = await w3.eth.send_raw_transaction(
    w3.eth.account.sign_transaction(tx, private_key).raw_transaction
)
tx_receipt = await w3.eth.get_transaction_receipt(tx_hash)

In [ ]:
"""Transfer"""
tx = await deployed_contract.functions.transfer(to=AsyncWeb3.to_checksum_address("0x70997970C51812dc3A010C7d01b50e0d17dc79C8"), value=1 * 10 ** 18).build_transaction(
    {
        "from": account.address,
        "nonce": await w3.eth.get_transaction_count(account.address),
    }
)
tx_hash = await w3.eth.send_raw_transaction(
    w3.eth.account.sign_transaction(tx, private_key).raw_transaction
)
tx_receipt = await w3.eth.get_transaction_receipt(tx_hash)

In [ ]:
"""Logs"""
import asyncio


async def transfer():
    for _ in range(3):
        await asyncio.sleep(1)
        tx = await deployed_contract.functions.transfer(
            to=AsyncWeb3.to_checksum_address(
                "0x70997970C51812dc3A010C7d01b50e0d17dc79C8"
            ),
            value=1 * 10**18,
        ).build_transaction(
            {
                "from": account.address,
                "nonce": await w3.eth.get_transaction_count(account.address),
            }
        )
        tx_hash = await w3.eth.send_raw_transaction(
            w3.eth.account.sign_transaction(tx, private_key).raw_transaction
        )
        tx_receipt = await w3.eth.get_transaction_receipt(tx_hash)


async def listen_to_logs():
    w3_ws = await get_new_ws_w3()
    transfer_topic = AsyncWeb3.keccak(text="Transfer(address,address,uint256)").hex()

    subscription_id = await w3_ws.eth.subscribe(
        "logs",
        {
            "address": deployed_addr,
            "topics": [transfer_topic],
        },
    )
    print("Subscribed to ERC20 transfers:", subscription_id)

    try:
        async with asyncio.timeout(30):  # listen for 30s
            async for message in w3_ws.socket.process_subscriptions():
                result = message.get("result")
                from_addr = "0x" + result["topics"][1].hex()[-40:]
                to_addr = "0x" + result["topics"][2].hex()[-40:]
                value = int(result["data"].hex(), 16) * 10 ** -18

                print(
                    f"Transfer | From: {from_addr} → To: {to_addr} | Value: {value}"
                )
    except asyncio.TimeoutError:
        print("No ERC20 transfers in time window — stopping.")


asyncio.gather(listen_to_logs(), transfer())